In [76]:
using ITensors
using CUDA
include("TTN_utilities.jl")
using .TTN_utilities
ITensors.set_warn_order(20)

20

### Basic Introduction

In [77]:
#= # Example of building a simple TTN manually
nsites = 4
sites = [Index(2, "site $i") for i in 1:nsites] =#

In ITensors when we define an index, we are implicitly defining a dimension for a generic vector space. In this case we are definig 4 bidimensional spaces (for example each one can be associated with a spin).

We can use indexes to define and work with tensors. For example defining: 

-- ITensor(index(2,a)) creates a rank-1 tensor of dimension 2, so a vector of lenght 2.

-- ITensor(index(2,a), index*(2,a)) creates a 2x2 matrix, so a transformation between the two 2d spaces


Contraction is a sum over indexes with same name.

In [78]:
#= # Leaf tensors
leaf1 = randomITensor(sites[1], sites[2])
leaf2 = randomITensor(sites[3], sites[4])

# Parent tensor
parent = randomITensor(sites[1], sites[2], sites[3], sites[4])

# contracting
result = parent * leaf1 * leaf2

println("Result of contracting the TTN with random vectors: ", result) =#

In [79]:
#= ABC = @visualize leaf1 * leaf2 * parent edge_labels=(tags=true,); =#

Contracting parent, a ITensor(i1,i2,i3,i4) with the two leaves, ITensor(i1,i2) and ITensor(i3,i4) returns a scalar since we are summing over all indexes.

# Tree-TensorNetwork

Following https://journals.aps.org/prb/abstract/10.1103/PhysRevB.99.155131

### Definition

<img src="./images/diagram_TTN.png" alt="Example Image" width="500"/>

As illustrated in Fig. 1(b), each circle represents
a tensor; each edge of the circle represents an individual
index of the tensor. The first tensor is a matrix connecting the
second and third tensors, while the remaining tensors are all
three-order tensors with three indices. The index between two
tensors is called a virtual bond, which would be contracted
hereafter. The left and right indices of the tensors in the
bottom of the TTN are respectively connected to two pixels
of the input image and hence are called physical bonds.

The bond dimension in a tensor network refers to the dimension of the shared index (or bond) connecting two tensors. 
Higher bond dimensions allow more complex entanglement between subsystems to be represented. In quantum many-body systems, the bond dimension represents the number of states used to approximate the entanglement structure.

Since bond dimension represents on how many indices we are summing, it is related to how well we can represent states with our tensor network.
In the context of Matrix Product States (MPS), the bond dimension directly corresponds to the Schmidt rank of the state for each bipartition. If the bond dimension is h , the MPS can capture states with a maximum Schmidt rank of h.

$|\psi\rangle=\sum_{i=1}^D \lambda_i\left|u_i\right\rangle_A \otimes\left|v_i\right\rangle_B$, where D is the Schmidt rank, the number of non-zero $\lambda$, which quantifies the enganglement between the two subsystems A and B (for D=1 the state is separable).

In [80]:
# Parameters
# Last layer contain only root tensor
number_physical_indexes = 8
bond_dimension = 2
number_layers = 3

# Define indexes
number_bound_1_indexes = div(number_physical_indexes,2)
number_bound_2_indexes = div(number_physical_indexes,4)

index_physical = [Index(2, "physical $i") for i in 1:number_physical_indexes]
index_bound_1 = [Index(bond_dimension, "bound 1 $i") for i in 1:number_bound_1_indexes]
index_bound_2 = [Index(bond_dimension, "bound 2 $i") for i in 1:number_bound_2_indexes]

# Build TTN
TTN = Vector{Vector{ITensor}}(undef, number_layers)
TTN[1] = [randomITensor(index_physical[2*i-1], index_physical[2*i], index_bound_1[i]) for i in 1:number_bound_1_indexes]
TTN[2] = [randomITensor(index_bound_1[2*i-1], index_bound_1[2*i], index_bound_2[i]) for i in 1:number_bound_2_indexes]
TTN[3] = [randomITensor(index_bound_2[1], index_bound_2[2])]

1-element Vector{ITensor}:
 ITensor ord=2
Dim 1: (dim=2|id=813|"bound21")
Dim 2: (dim=2|id=237|"bound22")
NDTensors.Dense{Float64, Vector{Float64}}
 2×2
  0.7078771135698944  -1.042245894005896
 -0.5248477493998434  -1.5737037049256306

In [81]:
# Converting tensors to CUDA tensors
v1 = TTN[1]
v2 = TTN[2]
v3 = TTN[3]
v1 = [ cu(t) for t in v1 ]
v2 = [ cu(t) for t in v2 ]
v3 = [ cu(t) for t in v3 ]
TTN[1] = v1
TTN[2] = v2
TTN[3] = v3
println("TTN converted to CUDA tensors")

TTN converted to CUDA tensors


In [ ]:
# calcluating norm at this stage with different contraction orders for future observation

contracted_initial_bottom = (TTN[1][1]*TTN[2][1]*TTN[1][2])*TTN[3][1]*(TTN[1][3]*TTN[2][2]*TTN[1][4])
norm_initial_bottom = sqrt((contracted_initial_bottom*dag(contracted_initial_bottom))[][])

contracted_initial_top = TTN[3][1]*TTN[2][1]*TTN[2][2]*TTN[1][1]*TTN[1][2]*TTN[1][3]*TTN[1][4]
norm_initial_top = sqrt((contracted_initial_top*dag(contracted_initial_top))[][])

tensors_ordered_array = vcat(TTN[1],TTN[2],TTN[3])
conj_tensors_ordered_array = [dag(t) for t in tensors_ordered_array]
norm_ordered_squared = 1.0
for (tensors_ordered_array, conj_tensors_ordered_array) in zip(tensors_ordered_array, conj_tensors_ordered_array)
    norm_ordered_squared *= tensors_ordered_array * conj_tensors_ordered_array
end
norm_initial_ordered = sqrt(norm_ordered_squared[][])

548.6406609072357

### Canonicalization basics

To simplify things, we need to bring our tensors in canonical form, in which each tensor is in canonical form for an index (in following case for index $\alpha_2$)
$$
\sum_{\alpha_4, \alpha_5} T_{\alpha_2, \alpha_4, \alpha_5}^{[2]} T_{\alpha_2^{\prime}, \alpha_4, \alpha_5}^{[2]}=\delta_{\alpha_2, \alpha_2^{\prime}},
$$

We can do so, by using QR decomposition. Starting from the leaves, we can decompose each tensor in a Unitary part and a Residual part. Then propagating the residual part towards the root we can for a "central tensor" containing all non-canonical information.

In [83]:
#= # QR decomposition of a tensor as example
# Example of building a simple TTN manually
nsites = 3
sites = [Index(2, "site $i") for i in 1:nsites]

A = randomITensor(sites[1], sites[2], sites[3])
A = A / norm(A)
Q, R = qr(A, [sites[1], sites[2]], [sites[3]])
println("norma di Q: ", norm(Q))
println("norma di R: ", norm(R)) =#

In [84]:
#= A_reconstructed = Q * R
println("Reconstructed tensor: ", A_reconstructed)
println("Reconstruction error: ", norm(A - A_reconstructed))
 =#

In [85]:
#= # Check that Q is unitary
# Normalize Q
Q = Q
Q_dagger = dag(Q)
result = Q * Q_dagger
println("Q * Q_dagger: ", result) =#

### Canonicalization of TTN

We can now procede in canonicalization procedure for each layer of the TNN, taking the root tensor as central tensor. In this procedure we have to keep an eye on indexes, since the QR decomposition, the first block of indices is assigned to Q and the second one (in our case, the only "upper" one) is assigned to R, to be contracted with the next layer. The algorithm creates a new set of indexes beetween the first and second layer, that we have to rename.

In [86]:
println("Starting reduction to canonical form")

# first layer
for i in 1:number_bound_1_indexes
    Q, R = qr(TTN[1][i], [index_physical[2*i-1],index_physical[2*i]], [index_bound_1[i]])
    norm_Q = sqrt((Q*dag(Q))[][])
    R = R*norm_Q
    Q = Q/norm_Q
    TTN[1][i] = Q
    TTN[2][Int(ceil(i/2))] = R*TTN[2][Int(ceil(i/2))]
end

# restoring indexes names
for i in 1:number_bound_1_indexes
    replaceind!(TTN[1][i], inds(TTN[1][i])[3], index_bound_1[i])
end
for i in 1:number_bound_2_indexes
    replaceind!(TTN[2][i], inds(TTN[2][i])[1], index_bound_1[2*i-1])
    replaceind!(TTN[2][i], inds(TTN[2][i])[2], index_bound_1[2*i])
end

# second layer
for i in 1:number_bound_2_indexes
    Q, R = qr(TTN[2][i], [index_bound_1[2*i-1],index_bound_1[2*i]], [index_bound_2[i]])
    norm_Q = sqrt((Q*dag(Q))[][])
    R = R*norm_Q
    Q = Q/norm_Q
    TTN[2][i] = Q
    TTN[3][1] = R * TTN[3][1]
end

# restoring indexes names
for i in 1:number_bound_2_indexes
    replaceind!(TTN[2][i], inds(TTN[2][i])[3], index_bound_2[i])
end
replaceind!(TTN[3][1], inds(TTN[3][1])[1], index_bound_2[1])
replaceind!(TTN[3][1], inds(TTN[3][1])[2], index_bound_2[2])

Starting reduction to canonical form


ITensor ord=2 (dim=2|id=813|"bound21") (dim=2|id=237|"bound22")
NDTensors.Dense{Float32, CuArray{Float32, 1, CUDA.DeviceMemory}}

Let's check the squared norm of the entire TTN making it contract with its conjugate (function defined in TTN_utilities)

In [ ]:
# calcluating norm at this stage with different contraction orders for future observation

contracted_final_bottom = (TTN[1][1]*TTN[2][1]*TTN[1][2])*TTN[3][1]*(TTN[1][3]*TTN[2][2]*TTN[1][4])
norm_final_bottom = sqrt((contracted_final_bottom*dag(contracted_final_bottom))[][])

contracted_final_top = TTN[3][1]*TTN[2][1]*TTN[2][2]*TTN[1][1]*TTN[1][2]*TTN[1][3]*TTN[1][4]
norm_final_top = sqrt((contracted_final_top*dag(contracted_final_top))[][])

norm_final_root = sqrt((TTN[3][1]*dag(TTN[3][1]))[][])

tensors_ordered_array = vcat(TTN[1],TTN[2],TTN[3])
conj_tensors_ordered_array = [dag(t) for t in tensors_ordered_array]
norm_ordered_squared = 1.0
for (tensors_ordered_array, conj_tensors_ordered_array) in zip(tensors_ordered_array, conj_tensors_ordered_array)
    norm_ordered_squared *= tensors_ordered_array * conj_tensors_ordered_array
end
norm_final_ordered = sqrt(norm_ordered_squared[][])

28.375807f0

### Observation about contraction order

If we contract the tensor network bottom up (or top bottom, same result) into a singe tensor and with its conjugate, to calculate norm, before the transformation to canonical form we get:

In [94]:
println("norm_initial_bottom: ",norm_initial_bottom)
println("norm_initial_top: ",norm_initial_top)

norm_initial_bottom: 28.375814
norm_initial_top: 28.375813


After the transformation:

In [95]:
println("norm_final_bottom: ",norm_final_bottom)
println("norm_final_top: ",norm_final_top)

norm_final_bottom: 28.375809
norm_final_top: 28.375807


This quantity is then conserved by the transformation. This is what we were expecting, since we are contracting back the $Q_i$ tensors back in the root contracted with all the $R_i$, finding again the initial tensor.

Now, if we contract each tensor with the respective conjugate before and after the transformation, we get another scalar value, that is not conserved by the procedure. However, after the transformation this value equals to the norm of the root tensor, and this is what we were expecting since each node different from the root is a unitary matrix.

In [97]:
println("norm_initial_ordered: ",norm_initial_ordered)
println("norm_final_ordered: ",norm_final_ordered)
println("norm_final_root: ",norm_final_root)

norm_initial_ordered: 548.6406609072357
norm_final_ordered: 227.00647693742675
norm_final_root: 227.00647
